<center>

# $\textbf{Merge Files}$

<center>

### $\textbf{Code}$

In [34]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col
import os
import time

In [35]:
inicio = time.time()
os.environ['PYSPARK_SUBMIT_ARGS'] = '--conf "spark.executor.extraJavaOptions=-XX:ReservedCodeCacheSize=2048m" --conf "spark.driver.extraJavaOptions=-XX:ReservedCodeCacheSize=2048m" pyspark-shell'

In [36]:
spark = SparkSession.builder.appName('MergeFiles').master("local").enableHiveSupport().getOrCreate()
spark

In [37]:
# Set paths to the Parquet files
file_paths = ["FilesParquet/Covid.parquet", "FilesParquet/GDP.parquet", "FilesParquet/Inflation.parquet",
              "FilesParquet/Migration.parquet", "FilesParquet/Population.parquet", "FilesParquet/Tax.parquet", "FilesParquet/Unemployment.parquet"]

# Read Parquet files into DataFrames
dfs = [spark.read.parquet(file_path) for file_path in file_paths]

# Merge or outer join the DataFrames
merged_df = dfs[0]

for df in dfs[1:]:
    merged_df = merged_df.join(df, on=["country","year"], how="outer")

### $\textbf{Pre-Processing}$

In [38]:
replacements = {
    "Korea": "North Korea",
    "Pacific island small states": "Pacific Islands"
}

# Rename columns based on replacements dictionary
for old_value, new_value in replacements.items():
    merged_df = merged_df.withColumnRenamed(old_value, new_value)

In [39]:
# Drop the specified columns
columns_to_drop = ['t. Lucia',
                   'ali', 
                   'alau',
                   'Anguilla',
                   'Bonaire Sint Eustatius and Saba',
                   'Cook Islands',
                   'American Samoa',
                   'Arab World',
                   'European Union',
                   'Faeroe Islands',
                   'Falkland Islands',
                   'Faroe Islands',
                   'French Guiana',
                   'Guadeloupe',
                   'Guernsey',
                   'High income',
                   'Isle of Man',
                   'Jersey',
                   'Kyrgyzstan',
                   'Late-demographic dividend',
                   'Latin America & Caribbean',
                   'Low income',
                   'Macao',              
                   'Martinique',
                   'Mayotte',
                   'Montserrat',
                   'Niue',
                   'North America',
                   'North Korea',
                   'Northern Cyprus',
                   'Northern Ireland',
                   'Oceania',
                   'Western Sahara',
                   'Wales',
                   'Wallis and Futuna',
                   'Vatican',
                   'Tokelau',
                   'Sub-Saharan Africa',
                   'Sub-Saharan Africa (Region) ',
                   'St. Lucia',
                   'St. Martin',
                   'Sint Maarten',
                   'Sint Maarten (Dutch part)',
                   'Scotland',
                   'Saint Martin',
                   'Saint Lucia',
                   'Saint Helena',
                   'Reunion',
                   'Post-demographic dividend',
                   'Pitcairn',
                   'Palestine',
                   'England',
                   'Saint Pierre and Miquelon',
                   'Saint Barthelemy',
                   'South America',
                   'St. Martin (French part)'
                   ]

# Drop the specified rows
merged_df = merged_df.filter(~df['country'].isin(columns_to_drop))

In [40]:
# Rename "no data" to "nan"
merged_df = merged_df.withColumn("country", when(df["country"] == "no data", "nan").otherwise(df["country"]))
merged_df = merged_df.withColumn("covid_cases", when(col("covid_cases").isNull(), 0).otherwise(col("covid_cases")))

In [41]:
# Sort the merged data by country and then by year
merged_df = merged_df.orderBy(col('country'), col('year'))

In [42]:
#Storing this dataframe in parquet
merged_df.write.mode("overwrite").parquet("FinalOuterParquet")
spark.read.parquet("FinalOuterParquet").show()
spark.stop()

+-----------+----+-----------+--------+---------+---------+----------+----------------+------------+
|    country|year|covid_cases|     gdp|inflation|migration|population|             tax|unemployment|
+-----------+----+-----------+--------+---------+---------+----------+----------------+------------+
|Afghanistan|2011|          0|   714.7|     11.8|   418796|  29249157|8.91679399595736|       7.918|
|Afghanistan|2012|          0| 784.611|      6.4|   105905|  30466479|7.70780060287773|       7.914|
|Afghanistan|2013|          0| 754.402|      7.4|    48076|  31541209|7.12277329020889|       7.914|
|Afghanistan|2014|          0| 746.922|      4.7|   255611|  32716210|6.88210294277301|        7.91|
|Afghanistan|2015|          0| 705.597|     -0.7|  -281739|  33753499|7.58538233129798|       9.002|
|Afghanistan|2016|          0| 617.126|      4.4|   -90238|  34636207|9.50265278288094|      10.092|
|Afghanistan|2017|          0| 635.789|      5.0|   -47090|  35643418|9.89845089688421|    

In [43]:
fim = time.time()
final = fim - inicio
print(final)

1.717498779296875
